In [59]:
# All the imports

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os


from numpy import random, delete
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

# import mord

In [60]:
dogs_train_df = pd.read_csv('train_dogs.csv')
cats_train_df = pd.read_csv('train_cats.csv')
dogs_test_df = pd.read_csv('eval_dogs.csv')
cats_test_df = pd.read_csv('eval_cats.csv')
dogs_train_df = dogs_train_df.loc[dogs_train_df['Gender']!=3]
cats_train_df = cats_train_df.loc[cats_train_df['Gender']!=3]

In [61]:
dogs_train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,PetID,PhotoAmt,AdoptionSpeed,DescLength,DescNumWords,DescPolite,DescNumAdj,DescNumVerb,DescCompound,AdoptedOrNot
0,1,Cocoa,17,117,307,2,3,0,0,2,...,382d948b1,2.0,3,248,43,1,5,10,0.8458,1
2,1,Spot,10,307,0,1,1,0,0,2,...,ffd2e82a9,3.0,3,271,53,0,6,11,0.9353,1
3,1,Daisy,4,307,0,2,5,0,0,1,...,435f8cb3d,0.0,4,33,6,1,1,1,0.8020,0
4,1,Shiro,24,141,307,1,4,5,7,2,...,a75703051,3.0,3,111,21,0,3,2,0.5994,1
5,1,3 Puppies,4,307,307,2,1,4,7,2,...,eea42fc56,0.0,4,153,26,1,4,7,0.9360,0


In [62]:
cats_train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,PetID,PhotoAmt,AdoptionSpeed,DescLength,DescNumWords,DescPolite,DescNumAdj,DescNumVerb,DescCompound,AdoptedOrNot
0,2,Grey & Midnight,2,266,0,2,1,2,6,2,...,88f092c58,9.0,3,258,46,0,7,7,0.8658,1
1,2,Samz,1,265,0,1,1,7,0,1,...,1b36762d2,13.0,1,498,103,2,13,25,0.9684,1
2,2,Tuty,2,266,0,2,1,7,0,1,...,4de32c463,2.0,1,198,39,0,2,11,0.1779,1
3,2,Xiao Bai And Baby,10,266,266,2,7,0,0,2,...,0ce98d199,1.0,3,465,88,0,8,16,0.5106,1
6,2,Orange,4,266,0,1,4,0,0,2,...,da8c280f1,6.0,3,69,13,0,1,2,0.8620,1


In [63]:
# Select certain features of interest in dogs and cats
    
# want Age, Breed1, Gender, Color1, Color2?, Color3?, MaturitySize, FurLength, Vaccinated, Dewormed, Sterilized, Health, Fee?, State, PhotoAmt
dogs_sub_train_df = dogs_train_df[['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt']].reset_index(drop=True)
cats_sub_train_df = cats_train_df[['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt']].reset_index(drop=True)

# test with the preprocessed split
dogs_sub_test_df = dogs_test_df[['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt']].reset_index(drop=True)
cats_sub_test_df = cats_test_df[['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt']].reset_index(drop=True)

print('Dog train size: ', dogs_sub_train_df.shape)
print('Dog test size: ', dogs_sub_test_df.shape)
print('Cat train size: ', cats_sub_train_df.shape)
print('Cat test size: ', cats_sub_test_df.shape)

# BINARY Training Labels
dogs_train_labels_df = dogs_train_df[['AdoptedOrNot']].reset_index(drop=True)
cats_train_labels_df = cats_train_df[['AdoptedOrNot']].reset_index(drop=True)

# BINARY Test Labels
dogs_test_labels_df = dogs_test_df[['AdoptedOrNot']].reset_index(drop=True)
cats_test_labels_df = cats_test_df[['AdoptedOrNot']].reset_index(drop=True)

print('# Dog train labels,: ', dogs_train_labels_df.shape)
print('# Dog test labels,: ', dogs_test_labels_df.shape)
print('# Cat train labels,: ', cats_train_labels_df.shape)
print('# Cat test labels,: ', cats_test_labels_df.shape)

Dog train size:  (4067, 11)
Dog test size:  (2992, 11)
Cat train size:  (3110, 11)
Cat test size:  (2498, 11)
# Dog train labels,:  (4067, 1)
# Dog test labels,:  (2992, 1)
# Cat train labels,:  (3110, 1)
# Cat test labels,:  (2498, 1)


In [6]:
# Making Categorical Variables into one-hot encodings - Color (check labels w/ csv)

dogs_sub_train_df.rename(columns={'Color1':'Color'}, inplace=True)
cats_sub_train_df.rename(columns={'Color1':'Color'}, inplace=True)
dogs_sub_test_df.rename(columns={'Color1':'Color'}, inplace=True)
cats_sub_test_df.rename(columns={'Color1':'Color'}, inplace=True)

for i in range(1,8):
    dogs_sub_train_df[f'Color{i}'] = dogs_sub_train_df['Color'].apply(lambda x: 1 if x == i else 0)
    cats_sub_train_df[f'Color{i}'] = cats_sub_train_df['Color'].apply(lambda x: 1 if x == i else 0)
    dogs_sub_test_df[f'Color{i}'] = dogs_sub_test_df['Color'].apply(lambda x: 1 if x == i else 0)
    cats_sub_test_df[f'Color{i}'] = cats_sub_test_df['Color'].apply(lambda x: 1 if x == i else 0)

dogs_sub_train_df.drop(columns=['Color'], inplace=True)
cats_sub_train_df.drop(columns=['Color'], inplace=True)
dogs_sub_test_df.drop(columns=['Color'], inplace=True)
cats_sub_test_df.drop(columns=['Color'], inplace=True)

In [7]:
dogs_sub_train_df.head()

,Age,Breed1,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,17,117,2,2,1,1,1,1,1,2.0,0,0,1,0,0,0,0
1,10,307,1,2,1,1,2,3,1,3.0,1,0,0,0,0,0,0
2,4,307,2,1,1,2,2,2,1,0.0,0,0,0,0,1,0,0
3,24,141,1,2,1,3,3,1,1,3.0,0,0,0,1,0,0,0
4,4,307,2,2,1,2,2,2,1,0.0,1,0,0,0,0,0,0


In [8]:
cats_sub_train_df.head()

,Age,Breed1,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,2,266,2,2,1,1,1,2,1,9.0,1,0,0,0,0,0,0
1,1,265,1,1,1,2,1,2,1,13.0,1,0,0,0,0,0,0
2,2,266,2,1,1,2,1,2,2,2.0,1,0,0,0,0,0,0
3,10,266,2,2,2,1,1,1,1,1.0,0,0,0,0,0,0,1
4,4,266,1,2,1,2,2,2,1,6.0,0,0,0,1,0,0,0


In [9]:
# Making Categorical Variables into one-hot encodings - Gender (0 for male, 1 for female - shift by 1 from dataset)

dogs_sub_train_df['Gender'] = dogs_sub_train_df['Gender'].apply(lambda x: 1 if x == 2 else 0)
cats_sub_train_df['Gender'] = cats_sub_train_df['Gender'].apply(lambda x: 1 if x == 2 else 0)
dogs_sub_test_df['Gender'] = dogs_sub_train_df['Gender'].apply(lambda x: 1 if x == 2 else 0)
cats_sub_test_df['Gender'] = cats_sub_train_df['Gender'].apply(lambda x: 1 if x == 2 else 0)

In [10]:
dogs_sub_train_df.head()

,Age,Breed1,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,17,117,1,2,1,1,1,1,1,2.0,0,0,1,0,0,0,0
1,10,307,0,2,1,1,2,3,1,3.0,1,0,0,0,0,0,0
2,4,307,1,1,1,2,2,2,1,0.0,0,0,0,0,1,0,0
3,24,141,0,2,1,3,3,1,1,3.0,0,0,0,1,0,0,0
4,4,307,1,2,1,2,2,2,1,0.0,1,0,0,0,0,0,0


In [11]:
cats_sub_train_df.head()

,Age,Breed1,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,2,266,1,2,1,1,1,2,1,9.0,1,0,0,0,0,0,0
1,1,265,0,1,1,2,1,2,1,13.0,1,0,0,0,0,0,0
2,2,266,1,1,1,2,1,2,2,2.0,1,0,0,0,0,0,0
3,10,266,1,2,2,1,1,1,1,1.0,0,0,0,0,0,0,1
4,4,266,0,2,1,2,2,2,1,6.0,0,0,0,1,0,0,0


In [12]:
# Version with Breeds
# Making Categorical Variables into one-hot encodings - Breed 

dogs_breed_train_df = dogs_sub_train_df.rename(columns={'Breed1':'Breed'})
cats_breed_train_df = cats_sub_train_df.rename(columns={'Breed1':'Breed'})
dogs_breed_test_df = dogs_sub_test_df.rename(columns={'Breed1':'Breed'})
cats_breed_test_df = cats_sub_test_df.rename(columns={'Breed1':'Breed'})

for i in range(1,308):
    dogs_breed_train_df[f'Breed{i}'] = dogs_breed_train_df['Breed'].apply(lambda x: 1 if x == i else 0)
    cats_breed_train_df[f'Breed{i}'] = cats_breed_train_df['Breed'].apply(lambda x: 1 if x == i else 0)
    dogs_breed_test_df[f'Breed{i}'] = dogs_breed_test_df['Breed'].apply(lambda x: 1 if x == i else 0)
    cats_breed_test_df[f'Breed{i}'] = cats_breed_test_df['Breed'].apply(lambda x: 1 if x == i else 0)

dogs_breed_train_df = dogs_breed_train_df.drop(columns=['Breed'])
cats_breed_train_df = cats_breed_train_df.drop(columns=['Breed']) 
dogs_breed_test_df = dogs_breed_test_df.drop(columns=['Breed'])
cats_breed_test_df = cats_breed_test_df.drop(columns=['Breed']) 

In [13]:
print(dogs_breed_train_df.shape)
print(dogs_breed_test_df.shape)
dogs_breed_train_df.head()

(4067, 323)
(2992, 323)


,Age,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,...,Breed298,Breed299,Breed300,Breed301,Breed302,Breed303,Breed304,Breed305,Breed306,Breed307
0,17,1,2,1,1,1,1,1,2.0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,0,2,1,1,2,3,1,3.0,1,...,0,0,0,0,0,0,0,0,0,1
2,4,1,1,1,2,2,2,1,0.0,0,...,0,0,0,0,0,0,0,0,0,1
3,24,0,2,1,3,3,1,1,3.0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,2,1,2,2,2,1,0.0,1,...,0,0,0,0,0,0,0,0,0,1


In [14]:
print(cats_breed_train_df.shape)
print(cats_breed_test_df.shape)
cats_breed_train_df.head()

(3110, 323)
(2498, 323)


,Age,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,...,Breed298,Breed299,Breed300,Breed301,Breed302,Breed303,Breed304,Breed305,Breed306,Breed307
0,2,1,2,1,1,1,2,1,9.0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,2,1,2,1,13.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2,1,1,1,2,1,2,2,2.0,1,...,0,0,0,0,0,0,0,0,0,0
3,10,1,2,2,1,1,1,1,1.0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,2,1,2,2,2,1,6.0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Version without Breeds
dogs_sub_train_df.drop(columns=['Breed1'], inplace=True)
cats_sub_train_df.drop(columns=['Breed1'], inplace=True)
dogs_sub_test_df.drop(columns=['Breed1'], inplace=True)
cats_sub_test_df.drop(columns=['Breed1'], inplace=True)

In [16]:
print(dogs_sub_train_df.shape)
print(dogs_sub_test_df.shape)
dogs_sub_train_df.head()

(4067, 16)
(2992, 16)


,Age,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,17,1,2,1,1,1,1,1,2.0,0,0,1,0,0,0,0
1,10,0,2,1,1,2,3,1,3.0,1,0,0,0,0,0,0
2,4,1,1,1,2,2,2,1,0.0,0,0,0,0,1,0,0
3,24,0,2,1,3,3,1,1,3.0,0,0,0,1,0,0,0
4,4,1,2,1,2,2,2,1,0.0,1,0,0,0,0,0,0


In [17]:
print(cats_sub_train_df.shape)
print(cats_sub_test_df.shape)
cats_sub_train_df.head()

(3110, 16)
(2498, 16)


,Age,Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,PhotoAmt,Color1,Color2,Color3,Color4,Color5,Color6,Color7
0,2,1,2,1,1,1,2,1,9.0,1,0,0,0,0,0,0
1,1,0,1,1,2,1,2,1,13.0,1,0,0,0,0,0,0
2,2,1,1,1,2,1,2,2,2.0,1,0,0,0,0,0,0
3,10,1,2,2,1,1,1,1,1.0,0,0,0,0,0,0,1
4,4,0,2,1,2,2,2,1,6.0,0,0,0,1,0,0,0


In [57]:
# PROCESSING INTO TRAIN AND TEST SETS 

# DOGS without breed
dogs_train = dogs_sub_train_df.values
dogs_train_labels = dogs_train_labels_df.values
dogs_test = dogs_sub_test_df.values
dogs_test_labels = dogs_test_labels_df.values

# DOGS with breed
dogs_breed_train = dogs_breed_train_df.values
dogs_breed_train_labels = dogs_train_labels_df.values
dogs_breed_test = dogs_breed_test_df.values
dogs_breed_test_labels = dogs_test_labels_df.values 

# CATS without breed
cats_train = cats_sub_train_df.values
cats_train_labels = cats_train_labels_df.values
cats_test = cats_sub_test_df.values
cats_test_labels = cats_test_labels_df.values 

# CATS with breed
cats_breed_train = cats_breed_train_df.values
cats_breed_train_labels = cats_train_labels_df.values
cats_breed_test = cats_breed_test_df.values
cats_breed_test_labels = cats_test_labels_df.values 

# START SCALING
scaler_d = StandardScaler()
#Fit on training set only.
scaler_d.fit(dogs_train)
#Apply transform to both the training set and the test set.
dogs_train = scaler_d.transform(dogs_train)
dogs_test = scaler_d.transform(dogs_test)

scaler_db = StandardScaler()
#Fit on training set only.
scaler_db.fit(dogs_breed_train)
#Apply transform to both the training set and the test set.
dogs_breed_train = scaler_db.transform(dogs_breed_train)
dogs_breed_test = scaler_db.transform(dogs_breed_test)


scaler_c = StandardScaler()
#Fit on training set only.
scaler_c.fit(cats_train)
#Apply transform to both the training set and the test set.
cats_train = scaler_c.transform(cats_train)
cats_test = scaler_c.transform(cats_test)

scaler_cb = StandardScaler()
#Fit on training set only.
scaler_cb.fit(cats_breed_train)
#Apply transform to both the training set and the test set.
cats_breed_train = scaler_cb.transform(cats_breed_train)
cats_breed_test = scaler_cb.transform(cats_breed_test)

print(dogs_train.shape)
print(dogs_test.shape)
print(dogs_test_labels.shape)
print(dogs_breed_train.shape)
print(dogs_breed_test.shape)
print(dogs_breed_test_labels.shape)

print(cats_train.shape)
print(cats_test.shape)
print(cats_test_labels.shape)
print(cats_breed_train.shape)
print(cats_breed_test.shape)
print(cats_breed_test_labels.shape)

(4067, 16)
(2992, 16)
(2992, 1)
(4067, 323)
(2992, 323)
(2992, 1)
(3110, 16)
(2498, 16)
(2498, 1)
(3110, 323)
(2498, 323)
(2498, 1)


In [19]:
# DOGS LOGISTIC REGRESSION

# after cv
log = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
parameters = {'C':[0.1, 1, 5, 10]}
clf = GridSearchCV(log, parameters, cv=5)
clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred = clf.predict(dogs_train)
adopt_pred = clf.predict(dogs_test)
lr_train_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
lr_test_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy w/o breed (after cv): ', lr_train_accuracy_dogs)
print('test accuracy w/o breed (after cv): ', lr_test_accuracy_dogs)
lr_f1_dogs = f1_score(dogs_test_labels, adopt_pred, average='macro')
print("f1 score w/o breed: ", lr_f1_dogs)
print(clf.best_params_)


train accuracy w/o breed (after cv):  0.7118268994344726
test accuracy w/o breed (after cv):  0.722927807486631
f1 score w/o breed:  0.44515437742918657
{'C': 0.1}


In [20]:
# using best params from prev cell to get weights
log2 = linear_model.LogisticRegression(C=0.1, solver='lbfgs', multi_class='ovr')
log2.fit(dogs_train,dogs_train_labels.ravel())
dogs_result = log2.coef_
print("Dogs log reg coefficients: ", log2.coef_)

Dogs log reg coefficients:  [[-0.18355633 -0.07756669 -0.02074365  0.22532602  0.19185249 -0.12405011
   0.12158453 -0.09419258  0.21162017  0.02686648 -0.06607477 -0.00458444
  -0.06992446  0.04491002  0.06309281  0.04500422]]


In [21]:
# CATS
# output for binary classification

# after cv
log = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
parameters = {'C':[0.1, 1, 5, 10]}
clf = GridSearchCV(log, parameters, cv=5)
clf.fit(cats_train,cats_train_labels.ravel())

train_pred = clf.predict(cats_train)
adopt_pred = clf.predict(cats_test)
lr_train_accuracy_cats = clf.score(cats_train, cats_train_labels)
lr_test_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', lr_train_accuracy_cats)
print('test accuracy (after cv): ', lr_test_accuracy_cats)
lr_f1_cats = f1_score(cats_test_labels, adopt_pred, average='macro')
print(lr_f1_cats)
print(clf.best_params_)


train accuracy (after cv):  0.7511254019292605
test accuracy (after cv):  0.7393915132105685
0.45333380396032613
{'C': 1}


In [22]:
# using best params from prev cell to get weights
log2 = linear_model.LogisticRegression(C=1, solver='lbfgs', multi_class='ovr')
log2.fit(cats_train,cats_train_labels.ravel())
cats_result = log2.coef_
print(log2.coef_)

[[-0.2703815  -0.08216308 -0.05228534  0.12372638  0.12899568 -0.28624018
   0.17126373 -0.06004336  0.41297661 -0.10292372  0.0638025   0.01826767
  -0.0416782   0.08264406 -0.00053789  0.07253714]]


In [23]:
print("FOR LOGISTIC REGRESSION WITHOUT BREEDS BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR LOGISTIC REGRESSION WITHOUT BREEDS BTWN DOGS AND CATS
[[-0.18355633 -0.07756669 -0.02074365  0.22532602  0.19185249 -0.12405011
   0.12158453 -0.09419258  0.21162017  0.02686648 -0.06607477 -0.00458444
  -0.06992446  0.04491002  0.06309281  0.04500422]]
[[-0.2703815  -0.08216308 -0.05228534  0.12372638  0.12899568 -0.28624018
   0.17126373 -0.06004336  0.41297661 -0.10292372  0.0638025   0.01826767
  -0.0416782   0.08264406 -0.00053789  0.07253714]]
[[ 0.08682517  0.00459639  0.0315417   0.10159965  0.06285681  0.16219007
  -0.0496792  -0.03414922 -0.20135643  0.12979021 -0.12987726 -0.02285211
  -0.02824627 -0.03773404  0.0636307  -0.02753291]]


In [24]:
# DOGS LOGISTIC REGRESSION WITH BREEDS

# after cv
log = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
parameters = {'C':[0.1, 1, 5, 10]}
clf = GridSearchCV(log, parameters, cv=5)
clf.fit(dogs_breed_train,dogs_breed_train_labels.ravel())

train_pred = clf.predict(dogs_breed_train)
adopt_pred = clf.predict(dogs_breed_test)
lr_btrain_accuracy_dogs = clf.score(dogs_breed_train, dogs_breed_train_labels)
lr_btest_accuracy_dogs = clf.score(dogs_breed_test, dogs_breed_test_labels)

print('train accuracy w/ breed (after cv): ', lr_btrain_accuracy_dogs)
print('test accuracy w/ breed (after cv): ', lr_btest_accuracy_dogs)
lr_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred, average='macro')
print("f1 score w/ breed: ", lr_bf1_dogs)
print(clf.best_params_)


train accuracy w/ breed (after cv):  0.7285468404229162
test accuracy w/ breed (after cv):  0.7286096256684492
f1 score w/ breed:  0.5086014328305076
{'C': 1}


In [25]:
# using best params from prev cell to get weights
log2 = linear_model.LogisticRegression(C=1, solver='lbfgs', multi_class='ovr')
log2.fit(dogs_breed_train,dogs_breed_train_labels.ravel())
dogs_result = log2.coef_
print("Dogs log reg coefficients: ", log2.coef_)

Dogs log reg coefficients:  [[-0.49299583 -0.06259352  0.01833682  0.14257785  0.24149128 -0.13796322
   0.13540011 -0.10692789  0.23482666  0.03889774 -0.04622743 -0.04187854
  -0.06683408  0.0323408   0.03723953  0.03735431 -0.11639011  0.
  -0.12358847  0.         -0.01087314  0.          0.          0.
   0.          0.188452   -0.12065156  0.          0.          0.
  -0.02431693 -0.11618755  0.1176457   0.1185819   0.22843056  0.16311631
   0.          0.          0.10695412  0.          0.08790903  0.03240289
   0.          0.          0.          0.          0.18476962  0.11352941
   0.          0.          0.          0.          0.          0.
   0.00795552  0.          0.         -0.01016179  0.         -0.00769516
   0.          0.          0.          0.          0.04992556  0.01177864
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.09456719  0.09635287  0.
   0.          0.          0.08984455  0.          0.     

In [26]:
# CATS
# output for binary classification

# after cv
log = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
parameters = {'C':[0.1, 1, 5, 10]}
clf = GridSearchCV(log, parameters, cv=5)
clf.fit(cats_breed_train,cats_breed_train_labels.ravel())

train_pred = clf.predict(cats_breed_train)
adopt_pred = clf.predict(cats_breed_test)
lr_btrain_accuracy_cats = clf.score(cats_breed_train, cats_breed_train_labels)
lr_btest_accuracy_cats = clf.score(cats_breed_test, cats_breed_test_labels)

print('train accuracy (after cv): ', lr_btrain_accuracy_cats)
print('test accuracy (after cv): ', lr_btest_accuracy_cats)
lr_bf1_cats = f1_score(cats_breed_test_labels, adopt_pred, average='macro')
print(lr_bf1_cats)
print(clf.best_params_)


train accuracy (after cv):  0.7585209003215434
test accuracy (after cv):  0.7373899119295436
0.4640165267414766
{'C': 0.1}


In [27]:
# using best params from prev cell to get weights
log2 = linear_model.LogisticRegression(C=0.1, solver='lbfgs', multi_class='ovr')
log2.fit(cats_breed_train,cats_breed_train_labels.ravel())
cats_result = log2.coef_
print(log2.coef_)

[[-0.3152463  -0.07816911 -0.0601994   0.01799913  0.12165239 -0.28286974
   0.15262039 -0.05607482  0.39387275 -0.09249612  0.05685872  0.0195164
  -0.04022089  0.06729422  0.00484549  0.06470251  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.05753994  0.          0.          0.         -0.09957865  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.0509038   0.          0.          0.        

In [29]:
print("FOR LOGISTIC REGRESSION WITH BREED BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR LOGISTIC REGRESSION WITH BREED BTWN DOGS AND CATS
[[-0.49299583 -0.06259352  0.01833682  0.14257785  0.24149128 -0.13796322
   0.13540011 -0.10692789  0.23482666  0.03889774 -0.04622743 -0.04187854
  -0.06683408  0.0323408   0.03723953  0.03735431 -0.11639011  0.
  -0.12358847  0.         -0.01087314  0.          0.          0.
   0.          0.188452   -0.12065156  0.          0.          0.
  -0.02431693 -0.11618755  0.1176457   0.1185819   0.22843056  0.16311631
   0.          0.          0.10695412  0.          0.08790903  0.03240289
   0.          0.          0.          0.          0.18476962  0.11352941
   0.          0.          0.          0.          0.          0.
   0.00795552  0.          0.         -0.01016179  0.         -0.00769516
   0.          0.          0.          0.          0.04992556  0.01177864
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.09456719  0.09635287  0.
   0.          0.          0.089

In [30]:
# TRAINING VIA Ridge Regression Classifier WITHOUT BREED - DOGS
rc = RidgeClassifier()
parameters = {'alpha':[0.01, 0.1, 1, 10], 'solver' : ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga')}
clf = GridSearchCV(rc, parameters, cv=5)

clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
rc_train_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
rc_test_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', rc_train_accuracy_dogs)
print('test accuracy (after cv): ', rc_test_accuracy_dogs)
rc_f1_dogs = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(rc_f1_dogs)
print(clf.best_params_)


train accuracy (after cv):  0.7118268994344726
test accuracy (after cv):  0.7209224598930482
0.4346954342229744
{'alpha': 0.01, 'solver': 'auto'}


In [31]:
# using best params from prev cell to get weights
rc2 = RidgeClassifier(alpha=0.01, solver='auto')
rc2.fit(dogs_train,dogs_train_labels.ravel())
dogs_result = rc2.coef_
print("Dogs ridge reg coefficients: ", rc2.coef_)

Dogs ridge reg coefficients:  [[-0.08007273 -0.03144034 -0.00905893  0.08932616  0.07710198 -0.05118785
   0.04907491 -0.04023068  0.07480436  0.01156013 -0.02628866 -0.00088246
  -0.0308806   0.01789303  0.02347017  0.01786619]]


In [32]:
# TRAINING VIA Ridge Regression Classifier WITHOUT BREED - CATS
rc = RidgeClassifier()
parameters = {'alpha':[0.01, 0.1, 1, 10], 'solver' : ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga')}
clf = GridSearchCV(rc, parameters, cv=5)

clf.fit(cats_train,cats_train_labels.ravel())

train_pred_cv = clf.predict(cats_train)
adopt_pred_cv = clf.predict(cats_test)
rc_train_accuracy_cats = clf.score(cats_train, cats_train_labels)
rc_test_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', rc_train_accuracy_cats)
print('test accuracy (after cv): ', rc_test_accuracy_cats)
rc_f1_cats = f1_score(cats_test_labels, adopt_pred_cv, average='macro')
print(rc_f1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.74983922829582
test accuracy (after cv):  0.7381905524419535
0.4419696106440625
{'alpha': 0.01, 'solver': 'auto'}


In [33]:
# using best params from prev cell to get weights
rc2 = RidgeClassifier(alpha=0.01, solver='auto')
rc2.fit(cats_train, cats_train_labels.ravel())
cats_result = rc2.coef_
print("Cats ridge reg coefficients: ", rc2.coef_)

Cats ridge reg coefficients:  [[-0.1132215  -0.02859744 -0.01708756  0.04289843  0.04075787 -0.09836661
   0.06048878 -0.02111663  0.10620242 -0.03608966  0.02181805  0.00647761
  -0.01288354  0.02766259  0.00123234  0.02358537]]


In [34]:
print("FOR RIDGE REGRESSION WITHOUT BREED BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR RIDGE REGRESSION WITHOUT BREED BTWN DOGS AND CATS
[[-0.08007273 -0.03144034 -0.00905893  0.08932616  0.07710198 -0.05118785
   0.04907491 -0.04023068  0.07480436  0.01156013 -0.02628866 -0.00088246
  -0.0308806   0.01789303  0.02347017  0.01786619]]
[[-0.1132215  -0.02859744 -0.01708756  0.04289843  0.04075787 -0.09836661
   0.06048878 -0.02111663  0.10620242 -0.03608966  0.02181805  0.00647761
  -0.01288354  0.02766259  0.00123234  0.02358537]]
[[ 0.03314877 -0.0028429   0.00802863  0.04642773  0.03634411  0.04717876
  -0.01141386 -0.01911404 -0.03139805  0.04764979 -0.0481067  -0.00736007
  -0.01799706 -0.00976956  0.02223782 -0.00571919]]


In [35]:
# TRAINING VIA Ridge Regression Classifier WITH BREED - DOGS
rc = RidgeClassifier()
parameters = {'alpha':[0.01, 0.1, 1, 10], 'solver' : ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga')}
clf = GridSearchCV(rc, parameters, cv=5)

clf.fit(dogs_breed_train,dogs_breed_train_labels.ravel())

train_pred_cv = clf.predict(dogs_breed_train)
adopt_pred_cv = clf.predict(dogs_breed_test)
rc_btrain_accuracy_dogs = clf.score(dogs_breed_train, dogs_breed_train_labels)
rc_btest_accuracy_dogs = clf.score(dogs_breed_test, dogs_breed_test_labels)

print('train accuracy (after cv): ', rc_btrain_accuracy_dogs)
print('test accuracy (after cv): ', rc_btest_accuracy_dogs)
rc_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
print(rc_bf1_dogs)
print(clf.best_params_)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


train accuracy (after cv):  0.726825670027047
test accuracy (after cv):  0.7279411764705882
0.4889814156595918
{'alpha': 0.01, 'solver': 'saga'}


In [36]:
# using best params from prev cell to get weights
rc2 = RidgeClassifier(alpha=0.01, solver='saga')
rc2.fit(dogs_breed_train,dogs_breed_train_labels.ravel())
dogs_result = rc2.coef_
print("Dogs ridge reg coefficients: ", rc2.coef_)

Dogs ridge reg coefficients:  [[-0.17138956 -0.0238517   0.00662669  0.05191537  0.09220923 -0.05501414
   0.05312128 -0.04412336  0.08029284  0.01646582 -0.01751274 -0.01620311
  -0.02860789  0.01268162  0.01013705  0.01430413 -0.02493712  0.
  -0.02769598  0.         -0.00746543  0.          0.          0.
   0.          0.02753401 -0.02637929  0.          0.          0.
  -0.01324506 -0.02498176  0.00760454  0.00824506  0.02363218  0.04054981
   0.          0.          0.01570021  0.          0.0074849   0.00337853
   0.          0.          0.          0.          0.01150359  0.00571979
   0.          0.          0.          0.          0.          0.
  -0.00237523  0.          0.         -0.00647324  0.         -0.00546469
   0.          0.          0.          0.          0.01382533  0.00087945
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.02341433  0.01084746  0.
   0.          0.          0.00829432  0.          0.   

In [37]:
# TRAINING VIA Ridge Regression Classifier WITH BREED - CATS
rc = RidgeClassifier()
parameters = {'alpha':[0.01, 0.1, 1, 10], 'solver' : ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga')}
clf = GridSearchCV(rc, parameters, cv=5)

clf.fit(cats_breed_train,cats_breed_train_labels.ravel())

train_pred_cv = clf.predict(cats_breed_train)
adopt_pred_cv = clf.predict(cats_breed_test)
rc_btrain_accuracy_cats = clf.score(cats_breed_train, cats_breed_train_labels)
rc_btest_accuracy_cats = clf.score(cats_breed_test, cats_breed_test_labels)

print('train accuracy (after cv): ', rc_btrain_accuracy_cats)
print('test accuracy (after cv): ', rc_btest_accuracy_cats)
rc_bf1_cats = f1_score(cats_breed_test_labels, adopt_pred_cv, average='macro')
print(rc_bf1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.7562700964630225
test accuracy (after cv):  0.7373899119295436
0.4576696602558671
{'alpha': 0.01, 'solver': 'auto'}


In [38]:
# using best params from prev cell to get weights
rc2 = RidgeClassifier(alpha=0.01, solver='auto')
rc2.fit(cats_breed_train, cats_breed_train_labels.ravel())
cats_result = rc2.coef_
print("Cats ridge reg coefficients: ", rc2.coef_)

Cats ridge reg coefficients:  [[-0.12482205 -0.02786546 -0.01995221  0.00336619  0.03946394 -0.0995852
   0.05602774 -0.02036359  0.10322218 -0.03230686  0.01895272  0.00749436
  -0.01285645  0.0221132   0.00294447  0.02147131  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.0081677   0.          0.          0.         -0.02943515  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.00480839  0.  

In [39]:
print("FOR RIDGE REGRESSION WITH BREED BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR RIDGE REGRESSION WITH BREED BTWN DOGS AND CATS
[[-0.17138956 -0.0238517   0.00662669  0.05191537  0.09220923 -0.05501414
   0.05312128 -0.04412336  0.08029284  0.01646582 -0.01751274 -0.01620311
  -0.02860789  0.01268162  0.01013705  0.01430413 -0.02493712  0.
  -0.02769598  0.         -0.00746543  0.          0.          0.
   0.          0.02753401 -0.02637929  0.          0.          0.
  -0.01324506 -0.02498176  0.00760454  0.00824506  0.02363218  0.04054981
   0.          0.          0.01570021  0.          0.0074849   0.00337853
   0.          0.          0.          0.          0.01150359  0.00571979
   0.          0.          0.          0.          0.          0.
  -0.00237523  0.          0.         -0.00647324  0.         -0.00546469
   0.          0.          0.          0.          0.01382533  0.00087945
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.02341433  0.01084746  0.
   0.          0.          0.008294

In [40]:
# TRAINING VIA NEAREST NEIGHBORS WITHOUT BREED - DOGS

neigh = KNeighborsClassifier()
parameters = {'n_neighbors':(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}
clf = GridSearchCV(neigh, parameters, cv=5)

clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
nn_train_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
nn_test_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', nn_train_accuracy_dogs)
print('test accuracy (after cv): ', nn_test_accuracy_dogs)
nn_f1_dogs = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(nn_f1_dogs)
print(clf.best_params_)


train accuracy (after cv):  0.7474797147774772
test accuracy (after cv):  0.704879679144385
0.5532753975091372
{'n_neighbors': 9}


In [41]:
# using best params from prev cell to get weights
neigh2 = KNeighborsClassifier(n_neighbors=9)
neigh2.fit(dogs_train,dogs_train_labels.ravel())
#dogs_result = neigh2.coef_
#print("Dogs nearest neighbors (9) coefficients: ", neigh2.coef_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=9, p=2,
           weights='uniform')

In [42]:
# TRAINING VIA NEAREST NEIGHBORS WITHOUT BREED - CATS

neigh = KNeighborsClassifier()
parameters = {'n_neighbors':(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}
clf = GridSearchCV(neigh, parameters, cv=5)

clf.fit(cats_train,cats_train_labels.ravel())

train_pred_cv = clf.predict(cats_train)
adopt_pred_cv = clf.predict(cats_test)
nn_train_accuracy_cats = clf.score(cats_train, cats_train_labels)
nn_test_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', nn_train_accuracy_cats)
print('test accuracy (after cv): ', nn_test_accuracy_cats)
nn_f1_cats = f1_score(cats_test_labels, adopt_pred_cv, average='macro')
print(nn_f1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.7790996784565917
test accuracy (after cv):  0.733386709367494
0.5373724183343862
{'n_neighbors': 9}


In [43]:
# using best params from prev cell to get weights
neigh2 = KNeighborsClassifier(n_neighbors=9)
neigh2.fit(cats_train,cats_train_labels.ravel())
#cats_result = neigh2.coef_
#print("Cats nearest neighbors (9) coefficients: ", neigh2.coef_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=9, p=2,
           weights='uniform')

In [44]:
# TRAINING VIA NEAREST NEIGHBORS WITH BREED - DOGS

neigh = KNeighborsClassifier()
parameters = {'n_neighbors':(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}
clf = GridSearchCV(neigh, parameters, cv=5)

clf.fit(dogs_breed_train,dogs_breed_train_labels.ravel())

train_pred_cv = clf.predict(dogs_breed_train)
adopt_pred_cv = clf.predict(dogs_breed_test)
nn_btrain_accuracy_dogs = clf.score(dogs_breed_train, dogs_breed_train_labels)
nn_btest_accuracy_dogs = clf.score(dogs_breed_test, dogs_breed_test_labels)

print('train accuracy (after cv): ', nn_btrain_accuracy_dogs)
print('test accuracy (after cv): ', nn_btest_accuracy_dogs)
nn_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
print(nn_bf1_dogs)
print(clf.best_params_)


train accuracy (after cv):  0.7683796410130317
test accuracy (after cv):  0.7242647058823529
0.5884012664114657
{'n_neighbors': 9}


In [45]:
# using best params from prev cell to get weights
neigh2 = KNeighborsClassifier(n_neighbors=9)
neigh2.fit(dogs_breed_train,dogs_breed_train_labels.ravel())
#dogs_result = neigh2.coef_
#print("Dogs nearest neighbors (9) coefficients: ", neigh2.coef_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=9, p=2,
           weights='uniform')

In [46]:
# TRAINING VIA NEAREST NEIGHBORS WITH BREED - CATS

neigh = KNeighborsClassifier()
parameters = {'n_neighbors':(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)}
clf = GridSearchCV(neigh, parameters, cv=5)

clf.fit(cats_breed_train,cats_breed_train_labels.ravel())

train_pred_cv = clf.predict(cats_breed_train)
adopt_pred_cv = clf.predict(cats_breed_test)
nn_btrain_accuracy_cats = clf.score(cats_breed_train, cats_breed_train_labels)
nn_btest_accuracy_cats = clf.score(cats_breed_test, cats_breed_test_labels)

print('train accuracy (after cv): ', nn_btrain_accuracy_cats)
print('test accuracy (after cv): ', nn_btest_accuracy_cats)
nn_bf1_cats = f1_score(cats_breed_test_labels, adopt_pred_cv, average='macro')
print(nn_bf1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.7771704180064308
test accuracy (after cv):  0.7369895916733387
0.5432109383154159
{'n_neighbors': 9}


In [47]:
# using best params from prev cell to get weights
neigh2 = KNeighborsClassifier(n_neighbors=9)
neigh2.fit(cats_breed_train,cats_breed_train_labels.ravel())
#cats_result = neigh2.coef_
#print("Cats nearest neighbors (9) coefficients: ", neigh2.coef_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=9, p=2,
           weights='uniform')

In [48]:
# TRAINING VIA DECISION TREES WITHOUT BREED - DOGS
dec_tree = DecisionTreeClassifier(random_state=0)

parameters = {'criterion':('gini', 'entropy')}
clf = GridSearchCV(dec_tree, parameters, cv=5)

clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
dt_train_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
dt_test_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', dt_train_accuracy_dogs)
print('test accuracy (after cv): ', dt_test_accuracy_dogs)
dt_f1_dogs = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(dt_f1_dogs)
print(clf.best_params_)


train accuracy (after cv):  0.9500860585197934
test accuracy (after cv):  0.6808155080213903
0.5993754016081572
{'criterion': 'entropy'}


In [49]:
# using best params from prev cell to get weights
dt2 = DecisionTreeClassifier(criterion='entropy')
dt2.fit(dogs_train,dogs_train_labels.ravel())
dogs_result = dt2.feature_importances_
print("Dogs decision tree feature importances ", dogs_result)

Dogs decision tree feature importances  [0.27002379 0.07015493 0.04682856 0.08561388 0.06431161 0.04856449
 0.06396817 0.0204817  0.20345968 0.0388773  0.01915228 0.01636566
 0.00726962 0.02376404 0.00486484 0.01629946]


In [50]:
# TRAINING VIA DECISION TREES WITHOUT BREED - CATS
dec_tree = DecisionTreeClassifier(random_state=0)

parameters = {'criterion':('gini', 'entropy')}
clf = GridSearchCV(dec_tree, parameters, cv=5)

clf.fit(cats_train,cats_train_labels.ravel())

train_pred_cv = clf.predict(cats_train)
adopt_pred_cv = clf.predict(cats_test)
dt_train_accuracy_cats = clf.score(cats_train, cats_train_labels)
dt_test_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', dt_train_accuracy_cats)
print('test accuracy (after cv): ', dt_test_accuracy_cats)
dt_f1_cats = f1_score(cats_test_labels, adopt_pred_cv, average='macro')
print(dt_f1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.9707395498392283
test accuracy (after cv):  0.6789431545236189
0.5809443726285052
{'criterion': 'gini'}


In [51]:
# using best params from prev cell to get weights
dt2 = DecisionTreeClassifier(criterion='gini')
dt2.fit(cats_train,cats_train_labels.ravel())
cats_result = dt2.feature_importances_
print("Cats decision tree feature importances ", cats_result)

Cats decision tree feature importances  [0.24140254 0.05677421 0.0658007  0.07863446 0.08471429 0.06007013
 0.06468778 0.01085474 0.21387398 0.0202288  0.02078126 0.01344279
 0.02100088 0.00556681 0.02974604 0.01242058]


In [52]:
print("FOR Decision Trees without breed BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR Decision Trees without breed BTWN DOGS AND CATS
[0.27002379 0.07015493 0.04682856 0.08561388 0.06431161 0.04856449
 0.06396817 0.0204817  0.20345968 0.0388773  0.01915228 0.01636566
 0.00726962 0.02376404 0.00486484 0.01629946]
[0.24140254 0.05677421 0.0658007  0.07863446 0.08471429 0.06007013
 0.06468778 0.01085474 0.21387398 0.0202288  0.02078126 0.01344279
 0.02100088 0.00556681 0.02974604 0.01242058]
[ 0.02862125  0.01338072 -0.01897214  0.00697942 -0.02040268 -0.01150564
 -0.00071961  0.00962695 -0.0104143   0.0186485  -0.00162899  0.00292287
 -0.01373126  0.01819723 -0.02488121  0.00387888]


In [53]:
# TRAINING VIA DECISION TREES WITH BREED - DOGS
dec_tree = DecisionTreeClassifier(random_state=0)

parameters = {'criterion':('gini', 'entropy')}
clf = GridSearchCV(dec_tree, parameters, cv=5)

clf.fit(dogs_breed_train,dogs_breed_train_labels.ravel())

train_pred_cv = clf.predict(dogs_breed_train)
adopt_pred_cv = clf.predict(dogs_breed_test)
dt_btrain_accuracy_dogs = clf.score(dogs_breed_train, dogs_breed_train_labels)
dt_btest_accuracy_dogs = clf.score(dogs_breed_test, dogs_breed_test_labels)

print('train accuracy (after cv): ', dt_btrain_accuracy_dogs)
print('test accuracy (after cv): ', dt_btest_accuracy_dogs)
dt_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
print(dt_bf1_dogs)
print(clf.best_params_)


train accuracy (after cv):  0.9564789771330219
test accuracy (after cv):  0.6734625668449198
0.5946947875040747
{'criterion': 'entropy'}


In [54]:
# using best params from prev cell to get weights
dt2 = DecisionTreeClassifier(criterion='entropy')
dt2.fit(dogs_breed_train,dogs_breed_train_labels.ravel())
dogs_result = dt2.feature_importances_
print("Dogs decision tree feature importances ", dogs_result)

Dogs decision tree feature importances  [0.22384402 0.0484747  0.04092704 0.05459457 0.05150081 0.05326474
 0.05805207 0.01621858 0.17463527 0.03487321 0.04024798 0.01294215
 0.01030488 0.01366078 0.00269122 0.01914075 0.         0.
 0.00208837 0.         0.00062893 0.         0.         0.
 0.         0.         0.00181387 0.         0.         0.
 0.00144587 0.00090811 0.         0.         0.00172994 0.00166698
 0.         0.         0.         0.         0.         0.00081876
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00040638 0.         0.
 0.         0.         0.00044262 0.         0.         0.
 0.00057914 0.         0.         0.00069901 0.         0.
 0.00089914 0.00371012 0.         0.0013194

In [55]:
# TRAINING VIA DECISION TREES WITH BREED - CATS
dec_tree = DecisionTreeClassifier(random_state=0)

parameters = {'criterion':('gini', 'entropy')}
clf = GridSearchCV(dec_tree, parameters, cv=5)

clf.fit(cats_breed_train,cats_breed_train_labels.ravel())

train_pred_cv = clf.predict(cats_breed_train)
adopt_pred_cv = clf.predict(cats_breed_test)
dt_btrain_accuracy_cats = clf.score(cats_breed_train, cats_breed_train_labels)
dt_btest_accuracy_cats = clf.score(cats_breed_test, cats_breed_test_labels)

print('train accuracy (after cv): ', dt_btrain_accuracy_cats)
print('test accuracy (after cv): ', dt_btest_accuracy_cats)
dt_bf1_cats = f1_score(cats_breed_test_labels, adopt_pred_cv, average='macro')
print(dt_bf1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.9829581993569132
test accuracy (after cv):  0.6821457165732586
0.5900472660199729
{'criterion': 'entropy'}


In [57]:
# using best params from prev cell to get weights
dt2 = DecisionTreeClassifier(criterion='entropy')
dt2.fit(cats_breed_train,cats_breed_train_labels.ravel())
cats_result = dt2.feature_importances_
print("Dogs decision tree feature importances ", cats_result)

Dogs decision tree feature importances  [0.20694113 0.06042016 0.06136445 0.05090739 0.06131244 0.04722563
 0.05279465 0.0127785  0.17225479 0.02174315 0.01899082 0.02137965
 0.01824273 0.01461917 0.0258974  0.01158486 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00100612 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.


In [58]:
print("FOR Decision Trees with breed BTWN DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR Decision Trees with breed BTWN DOGS AND CATS
[0.22384402 0.0484747  0.04092704 0.05459457 0.05150081 0.05326474
 0.05805207 0.01621858 0.17463527 0.03487321 0.04024798 0.01294215
 0.01030488 0.01366078 0.00269122 0.01914075 0.         0.
 0.00208837 0.         0.00062893 0.         0.         0.
 0.         0.         0.00181387 0.         0.         0.
 0.00144587 0.00090811 0.         0.         0.00172994 0.00166698
 0.         0.         0.         0.         0.         0.00081876
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00040638 0.         0.
 0.         0.         0.00044262 0.         0.         0.
 0.00057914 0.         0.         0.00069901 0.         0.
 0.00089914 0.00371012 0.         

In [59]:
# TRAINING VIA LINEAR SVM WITHOUT BREED - DOGS
svm = LinearSVC(random_state=0, tol=1e-5, max_iter=10000)

parameters = {'loss':('hinge', 'squared_hinge'), 'C':[0.1, 0.5, 1, 5, 10]}
clf = GridSearchCV(svm, parameters, cv=5)

clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
svm_train_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
svm_test_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', svm_train_accuracy_dogs)
print('test accuracy (after cv): ', svm_test_accuracy_dogs)
svm_f1_dogs = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(svm_f1_dogs)
print(clf.best_params_)



/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train accuracy (after cv):  0.7113351364642242
test accuracy (after cv):  0.7199197860962567
0.41857753595025265
{'C': 0.1, 'loss': 'hinge'}


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [60]:
# using best params from prev cell to get weights
svm2 = LinearSVC(random_state=0, tol=1e-5, max_iter=10000, C=0.1, loss='hinge')
svm2.fit(dogs_train,dogs_train_labels.ravel())
dogs_result = svm2.coef_
print("Dogs svm coef ", dogs_result)

Dogs svm coef  [[-4.22224181e-14 -6.94695326e-14 -1.11093044e-13  3.86587758e-14
   1.09704423e-13 -9.93412195e-14 -6.20877514e-14 -2.93305318e-14
  -5.61501094e-14  2.87950302e-14 -3.04162458e-14  1.70763247e-14
   5.98648502e-14 -1.41471956e-14 -3.03657293e-15 -4.41010924e-14]]


In [61]:
# TRAINING VIA LINEAR SVM WITHOUT BREED - CATS
svm = LinearSVC(random_state=0, tol=1e-5, max_iter=10000)

parameters = {'loss':('hinge', 'squared_hinge'), 'C':[0.1, 0.5, 1, 5, 10]}
clf = GridSearchCV(svm, parameters, cv=5)

clf.fit(cats_train,cats_train_labels.ravel())

train_pred_cv = clf.predict(cats_train)
adopt_pred_cv = clf.predict(cats_test)
svm_train_accuracy_cats = clf.score(cats_train, cats_train_labels)
svm_test_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', svm_train_accuracy_cats)
print('test accuracy (after cv): ', svm_test_accuracy_cats)
svm_f1_cats = f1_score(cats_test_labels, adopt_pred_cv, average='macro')
print(svm_f1_cats)
print(clf.best_params_)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train accuracy (after cv):  0.7533762057877813
test accuracy (after cv):  0.7397918334667735
0.4252185918085596
{'C': 0.1, 'loss': 'hinge'}


/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [62]:
# using best params from prev cell to get weights
svm2 = LinearSVC(random_state=0, tol=1e-5, max_iter=10000, C=0.1, loss='hinge')
svm2.fit(cats_train,cats_train_labels.ravel())
cats_result = svm2.coef_
print("Cast svm coef ", cats_result)

Cast svm coef  [[-2.35493193e-14 -3.54134444e-14  1.26321138e-15 -2.08103896e-14
  -1.06972407e-13  5.26925256e-14 -2.18828488e-14 -1.29909439e-14
  -6.61357348e-15 -1.02175265e-13  5.39918982e-14 -4.24613208e-14
   3.38054517e-14  9.18808028e-14  4.98097270e-14  2.57894754e-15]]


In [63]:
print("FOR SVM BTWN WITHOUT BREED DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR SVM BTWN WITHOUT BREED DOGS AND CATS
[[-4.22224181e-14 -6.94695326e-14 -1.11093044e-13  3.86587758e-14
   1.09704423e-13 -9.93412195e-14 -6.20877514e-14 -2.93305318e-14
  -5.61501094e-14  2.87950302e-14 -3.04162458e-14  1.70763247e-14
   5.98648502e-14 -1.41471956e-14 -3.03657293e-15 -4.41010924e-14]]
[[-2.35493193e-14 -3.54134444e-14  1.26321138e-15 -2.08103896e-14
  -1.06972407e-13  5.26925256e-14 -2.18828488e-14 -1.29909439e-14
  -6.61357348e-15 -1.02175265e-13  5.39918982e-14 -4.24613208e-14
   3.38054517e-14  9.18808028e-14  4.98097270e-14  2.57894754e-15]]
[[-1.86730988e-14 -3.40560882e-14 -1.12356255e-13  5.94691653e-14
   2.16676830e-13 -1.52033745e-13 -4.02049025e-14 -1.63395879e-14
  -4.95365359e-14  1.30970295e-13 -8.44081439e-14  5.95376456e-14
   2.60593985e-14 -1.06027998e-13 -5.28463000e-14 -4.66800399e-14]]


In [64]:
# TRAINING VIA LINEAR SVM WITH BREED - DOGS
svm = LinearSVC(random_state=0, tol=1e-5, max_iter=10000)

parameters = {'loss':('hinge', 'squared_hinge'), 'C':[0.1, 0.5, 1, 5, 10]}
clf = GridSearchCV(svm, parameters, cv=5)

clf.fit(dogs_breed_train,dogs_breed_train_labels.ravel())

train_pred_cv = clf.predict(dogs_breed_train)
adopt_pred_cv = clf.predict(dogs_breed_test)
svm_btrain_accuracy_dogs = clf.score(dogs_breed_train, dogs_breed_train_labels)
svm_btest_accuracy_dogs = clf.score(dogs_breed_test, dogs_breed_test_labels)

print('train accuracy (after cv): ', svm_btrain_accuracy_dogs)
print('test accuracy (after cv): ', svm_btest_accuracy_dogs)
svm_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
print(svm_bf1_dogs)
print(clf.best_params_)



/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train accuracy (after cv):  0.7273174329972953
test accuracy (after cv):  0.7272727272727273
0.4929348598145593
{'C': 0.1, 'loss': 'squared_hinge'}


In [65]:
# using best params from prev cell to get weights
svm2 = LinearSVC(random_state=0, tol=1e-5, max_iter=10000, C=0.1, loss='squared_hinge')
svm2.fit(dogs_breed_train,dogs_breed_train_labels.ravel())
dogs_result = svm2.coef_
print("Dogs svm coef ", dogs_result)

Dogs svm coef  [[-0.18496021 -0.0248037   0.00698737  0.05450267  0.09573289 -0.05706221
   0.05447228 -0.04408885  0.0841218   0.01670502 -0.01822665 -0.01629595
  -0.02887747  0.01312791  0.01248312  0.01374932 -0.02233463  0.
  -0.02534398  0.         -0.00384455  0.          0.          0.
   0.          0.03564525 -0.02398139  0.          0.          0.
  -0.00766446 -0.02241488  0.01160456  0.01172703  0.05152073  0.06012873
   0.          0.          0.01853217  0.          0.00997032  0.0143928
   0.          0.          0.          0.          0.01913924  0.0099187
   0.          0.          0.          0.          0.          0.
   0.00525062  0.          0.         -0.00218943  0.         -0.00102333
   0.          0.          0.          0.          0.02067587  0.00605035
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.03749173  0.01360125  0.
   0.          0.          0.01081576  0.          0.          0.
   0.05

In [66]:
# TRAINING VIA LINEAR SVM with Breed - CATS
svm = LinearSVC(random_state=0, tol=1e-5, max_iter=10000)

parameters = {'loss':('hinge', 'squared_hinge'), 'C':[0.1, 0.5, 1, 5, 10]}
clf = GridSearchCV(svm, parameters, cv=5)

clf.fit(cats_breed_train,cats_breed_train_labels.ravel())

train_pred_cv = clf.predict(cats_breed_train)
adopt_pred_cv = clf.predict(cats_breed_test)
svm_btrain_accuracy_cats = clf.score(cats_breed_train, cats_breed_train_labels)
svm_btest_accuracy_cats = clf.score(cats_breed_test, cats_breed_test_labels)

print('train accuracy (after cv): ', svm_btrain_accuracy_cats)
print('test accuracy (after cv): ', svm_btest_accuracy_cats)
svm_bf1_cats = f1_score(cats_breed_test_labels, adopt_pred_cv, average='macro')
print(svm_bf1_cats)
print(clf.best_params_)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


train accuracy (after cv):  0.7572347266881029
test accuracy (after cv):  0.7381905524419535
0.46060341924364584
{'C': 0.1, 'loss': 'squared_hinge'}


In [67]:
# using best params from prev cell to get weights
svm2 = LinearSVC(random_state=0, tol=1e-5, max_iter=10000, C=0.1, loss='squared_hinge')
svm2.fit(cats_breed_train,cats_breed_train_labels.ravel())
cats_result = svm2.coef_
print("Cast svm coef ", cats_result)

Cast svm coef  [[-0.12833843 -0.02767451 -0.02154018  0.00598784  0.04176678 -0.10100154
   0.0557145  -0.02041725  0.11815389 -0.03330744  0.02023949  0.00711882
  -0.01365533  0.02324763  0.00283284  0.02208076  0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.00802753  0.          0.          0.         -0.02939885  0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.00463392  0.          0.    

In [68]:
print("FOR SVM BTWN WITH BREED DOGS AND CATS")
print(dogs_result)
print(cats_result)
diff = dogs_result-cats_result
print(diff)

FOR SVM BTWN WITH BREED DOGS AND CATS
[[-0.18496021 -0.0248037   0.00698737  0.05450267  0.09573289 -0.05706221
   0.05447228 -0.04408885  0.0841218   0.01670502 -0.01822665 -0.01629595
  -0.02887747  0.01312791  0.01248312  0.01374932 -0.02233463  0.
  -0.02534398  0.         -0.00384455  0.          0.          0.
   0.          0.03564525 -0.02398139  0.          0.          0.
  -0.00766446 -0.02241488  0.01160456  0.01172703  0.05152073  0.06012873
   0.          0.          0.01853217  0.          0.00997032  0.0143928
   0.          0.          0.          0.          0.01913924  0.0099187
   0.          0.          0.          0.          0.          0.
   0.00525062  0.          0.         -0.00218943  0.         -0.00102333
   0.          0.          0.          0.          0.02067587  0.00605035
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.03749173  0.01360125  0.
   0.          0.          0.01081576  0.         

In [69]:
# COMPARE MODELS FOR DOGS NO BREED
print("logistic regression train accuracy: ", lr_train_accuracy_dogs)
print("logistic regression test accuracy: ", lr_test_accuracy_dogs)
print("logistic regression f1 score: ", lr_f1_dogs)

print("ridge regression train accuracy: ", rc_train_accuracy_dogs)
print("ridge regression test accuracy: ", rc_test_accuracy_dogs)
print("ridge regression f1 score: ", rc_f1_dogs)

print("nearest neighbors train accuracy: ", nn_train_accuracy_dogs)
print("nearest neighbors test accuracy: ", nn_test_accuracy_dogs)
print("nearest neighbors f1 score: ", nn_f1_dogs)

print("decision tree train accuracy: ", dt_train_accuracy_dogs)
print("decision tree test accuracy: ", dt_test_accuracy_dogs)
print("decision tree f1 score: ", dt_f1_dogs)

print("svm train accuracy: ", svm_train_accuracy_dogs)
print("svm test accuracy: ", svm_test_accuracy_dogs)
print("svm f1 score: ", svm_f1_dogs)

#print("DOGS: Choose logistic regression for highest test accuracy 0.7219 (f1 score 0.486), choose neighest neighbors for best balance w/ f1 score 0.719 test accuracy (f1 score 0.609)")

logistic regression train accuracy:  0.7118268994344726
logistic regression test accuracy:  0.722927807486631
logistic regression f1 score:  0.44515437742918657
ridge regression train accuracy:  0.7118268994344726
ridge regression test accuracy:  0.7209224598930482
ridge regression f1 score:  0.4346954342229744
nearest neighbors train accuracy:  0.7474797147774772
nearest neighbors test accuracy:  0.704879679144385
nearest neighbors f1 score:  0.5532753975091372
decision tree train accuracy:  0.9500860585197934
decision tree test accuracy:  0.6808155080213903
decision tree f1 score:  0.5993754016081572
svm train accuracy:  0.7113351364642242
svm test accuracy:  0.7199197860962567
svm f1 score:  0.41857753595025265


In [70]:
# COMPARE MODELS FOR DOGS WITH BREED
print("logistic regression train accuracy (with breed): ", lr_btrain_accuracy_dogs)
print("logistic regression test accuracy (with breed): ", lr_btest_accuracy_dogs)
print("logistic regression f1 score (with breed): ", lr_bf1_dogs)

print("ridge regression train accuracy: ", rc_btrain_accuracy_dogs)
print("ridge regression test accuracy: ", rc_btest_accuracy_dogs)
print("ridge regression f1 score: ", rc_bf1_dogs)

print("nearest neighbors train accuracy: ", nn_btrain_accuracy_dogs)
print("nearest neighbors test accuracy: ", nn_btest_accuracy_dogs)
print("nearest neighbors f1 score: ", nn_bf1_dogs)

print("decision tree train accuracy: ", dt_btrain_accuracy_dogs)
print("decision tree test accuracy: ", dt_btest_accuracy_dogs)
print("decision tree f1 score: ", dt_bf1_dogs)

print("svm train accuracy: ", svm_btrain_accuracy_dogs)
print("svm test accuracy: ", svm_btest_accuracy_dogs)
print("svm f1 score: ", svm_bf1_dogs)

#print("DOGS: Choose logistic regression for highest test accuracy 0.7219 (f1 score 0.486), choose neighest neighbors for best balance w/ f1 score 0.719 test accuracy (f1 score 0.609)")

logistic regression train accuracy (with breed):  0.7285468404229162
logistic regression test accuracy (with breed):  0.7286096256684492
logistic regression f1 score (with breed):  0.5086014328305076
ridge regression train accuracy:  0.726825670027047
ridge regression test accuracy:  0.7279411764705882
ridge regression f1 score:  0.4889814156595918
nearest neighbors train accuracy:  0.7683796410130317
nearest neighbors test accuracy:  0.7242647058823529
nearest neighbors f1 score:  0.5884012664114657
decision tree train accuracy:  0.9564789771330219
decision tree test accuracy:  0.6734625668449198
decision tree f1 score:  0.5946947875040747
svm train accuracy:  0.7273174329972953
svm test accuracy:  0.7272727272727273
svm f1 score:  0.4929348598145593


In [71]:
# COMPARE MODELS FOR CATS
print("logistic regression train accuracy (without breed(: ", lr_train_accuracy_cats)
print("logistic regression test accuracy: ", lr_test_accuracy_cats)
print("logistic regression f1 score: ", lr_f1_cats)

print("ridge regression train accuracy: ", rc_train_accuracy_cats)
print("ridge regression test accuracy: ", rc_test_accuracy_cats)
print("ridge regression f1 score: ", rc_f1_cats)

print("nearest neighbors train accuracy: ", nn_train_accuracy_cats)
print("nearest neighbors test accuracy: ", nn_test_accuracy_cats)
print("nearest neighbors f1 score: ", nn_f1_cats)

print("decision tree train accuracy: ", dt_train_accuracy_cats)
print("decision tree test accuracy: ", dt_test_accuracy_cats)
print("decision tree f1 score: ", dt_f1_cats)

print("svm train accuracy: ", svm_train_accuracy_cats)
print("svm test accuracy: ", svm_test_accuracy_cats)
print("svm f1 score: ", svm_f1_cats)

#print("CATS: Choose svm for highest test accuracy 0.7546 (f1 score 0.430), choose neighest neighbors for best balance w/ f1 score 0.741 test accuracy (f1 score 0.571)")

logistic regression train accuracy (without breed(:  0.7511254019292605
logistic regression test accuracy:  0.7393915132105685
logistic regression f1 score:  0.45333380396032613
ridge regression train accuracy:  0.74983922829582
ridge regression test accuracy:  0.7381905524419535
ridge regression f1 score:  0.4419696106440625
nearest neighbors train accuracy:  0.7790996784565917
nearest neighbors test accuracy:  0.733386709367494
nearest neighbors f1 score:  0.5373724183343862
decision tree train accuracy:  0.9707395498392283
decision tree test accuracy:  0.6789431545236189
decision tree f1 score:  0.5809443726285052
svm train accuracy:  0.7533762057877813
svm test accuracy:  0.7397918334667735
svm f1 score:  0.4252185918085596


In [72]:
# COMPARE MODELS FOR CATS
print("logistic regression train accuracy (with breed): ", lr_btrain_accuracy_cats)
print("logistic regression test accuracy: ", lr_btest_accuracy_cats)
print("logistic regression f1 score: ", lr_bf1_cats)

print("ridge regression train accuracy: ", rc_btrain_accuracy_cats)
print("ridge regression test accuracy: ", rc_btest_accuracy_cats)
print("ridge regression f1 score: ", rc_bf1_cats)

print("nearest neighbors train accuracy: ", nn_btrain_accuracy_cats)
print("nearest neighbors test accuracy: ", nn_btest_accuracy_cats)
print("nearest neighbors f1 score: ", nn_bf1_cats)

print("decision tree train accuracy: ", dt_btrain_accuracy_cats)
print("decision tree test accuracy: ", dt_btest_accuracy_cats)
print("decision tree f1 score: ", dt_bf1_cats)

print("svm train accuracy: ", svm_btrain_accuracy_cats)
print("svm test accuracy: ", svm_btest_accuracy_cats)
print("svm f1 score: ", svm_bf1_cats)

#print("CATS: Choose svm for highest test accuracy 0.7546 (f1 score 0.430), choose neighest neighbors for best balance w/ f1 score 0.741 test accuracy (f1 score 0.571)")

logistic regression train accuracy (with breed):  0.7585209003215434
logistic regression test accuracy:  0.7373899119295436
logistic regression f1 score:  0.4640165267414766
ridge regression train accuracy:  0.7562700964630225
ridge regression test accuracy:  0.7373899119295436
ridge regression f1 score:  0.4576696602558671
nearest neighbors train accuracy:  0.7771704180064308
nearest neighbors test accuracy:  0.7369895916733387
nearest neighbors f1 score:  0.5432109383154159
decision tree train accuracy:  0.9829581993569132
decision tree test accuracy:  0.6821457165732586
decision tree f1 score:  0.5900472660199729
svm train accuracy:  0.7572347266881029
svm test accuracy:  0.7381905524419535
svm f1 score:  0.46060341924364584


In [73]:
# THIS IS AN OLD VERSION NOT SURE IF IT WORKS WELL NOW NO BREED
# attempts at ensemble - voting classifier

clf1 = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
clf2 = DecisionTreeClassifier(random_state=0)
clf3 = KNeighborsClassifier()
eclf = VotingClassifier(estimators=[('lr', clf1), ('dtree', clf2), ('kneigh', clf3)], voting='hard')

params = {'lr__C': [1.0, 100.0], 'kneigh__n_neighbors': [2, 3, 4, 5]}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(dogs_train, dogs_train_labels.ravel())

train_pred_cv = grid.predict(dogs_train)
adopt_pred_cv = grid.predict(dogs_test)
train_accuracy_cv = grid.score(dogs_train, dogs_train_labels)
test_accuracy_cv = grid.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', train_accuracy_cv)
print('test accuracy (after cv): ', test_accuracy_cv)
f1_cv = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(f1_cv)
print(grid.best_params_)

train accuracy (after cv):  0.8421440865502827
test accuracy (after cv):  0.7195855614973262
0.5444982334547079
{'kneigh__n_neighbors': 5, 'lr__C': 1.0}


In [ ]:
## DEAD CODE IN CASE NEED LATER


# pre-processing, separate by animal type
train = pd.read_csv('train.csv')
#dogs_df

#print(train.shape)
#train = train.dropna()
#print(train.shape)

#dogs_df = train.loc[train['Type'] == 1]
#cats_df = train.loc[train['Type'] == 2]
#print(dogs_df.shape)
#print(cats_df.shape)

#cats_sub_train_df = cats_train_df[['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'State', 'PhotoAmt']].reset_index(drop=True)
# dogs_sub_train_df = dogs_train_df[['Age', 'Gender', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt']].reset_index(drop=True)

# function to change to binary class (adopted w/in 100 days 1, not 0)
def adoptSpeed(x):
    if x == 4: # might be too rare of a bench mark if it's 2/7 ?
        return 1
    else:
        return 0

# def adoptSpeedBins(x):
#     if x == 0 or x == 1: 
#         return 0
#     elif x == 2:
#         return 1
#     else:
#         return 2
    
#dogs_labels_2_df = dogs_labels_df.applymap(adoptSpeed) # train labels
#dogs_labels2_2_df = dogs_labels2_df.applymap(adoptSpeed) # test labels

# dogs_labels_bins_df = dogs_labels_df.applymap(adoptSpeedBins)
#cats_labels_2_df = cats_labels_df.applymap(adoptSpeed) # train labels
#cats_labels2_2_df = cats_labels2_df.applymap(adoptSpeed) # test labels

# cats_labels_bins_df = cats_labels_df.applymap(adoptSpeedBins)

#cats_train_bins_labels = cats_labels_bins_df.values

#dogs_train, dogs_test, dogs_train_labels, dogs_test_labels = train_test_split(dogs_train_orig, dogs_train_labels, test_size=0.40)
#print(dogs_train.shape)
#print(dogs_train_2_labels.shape)
#dogs_train_2, dogs_test_2, dogs_train_2_labels, dogs_test_2_labels = train_test_split(dogs_train_orig, dogs_train_2_labels, test_size=0.40)
#dogs_train_bins, dogs_test_bins, dogs_train_bins_labels, dogs_test_bins_labels = train_test_split(dogs_train_orig, dogs_train_bins_labels, test_size=0.40)


#cats_train, cats_test, cats_train_labels, cats_test_labels = train_test_split(cats_train_orig, cats_train_labels, test_size=0.40)
#cats_train_2, cats_test_2, cats_train_2_labels, cats_test_2_labels = train_test_split(cats_train_orig, cats_train_2_labels, test_size=0.40)
#cats_train_bins, cats_test_bins, cats_train_bins_labels, cats_test_bins_labels = train_test_split(cats_train_orig, cats_train_bins_labels, test_size=0.40)

#scaler_d2 = StandardScaler()
#Fit on training set only.
#scaler_d2.fit(dogs_train_2)
#Apply transform to both the training set and the test set.
#dogs_train_2 = scaler_d2.transform(dogs_train_2)
#dogs_test_2 = scaler_d2.transform(dogs_test_2)

#scaler_dbins = StandardScaler()
#Fit on training set only.
#scaler_dbins.fit(dogs_train_bins)
#Apply transform to both the training set and the test set.
#dogs_train_bins = scaler_dbins.transform(dogs_train_bins)
#dogs_test_bins = scaler_dbins.transform(dogs_test_bins)

#scaler_c2 = StandardScaler()
#Fit on training set only.
#scaler_c2.fit(cats_train_2)
#Apply transform to both the training set and the test set.
#cats_train_2 = scaler_c2.transform(cats_train_2)
#cats_test_2 = scaler_c2.transform(cats_test_2)

#scaler_cbins = StandardScaler()
#Fit on training set only.
#scaler_cbins.fit(cats_train_bins)
#Apply transform to both the training set and the test set.
#cats_train_bins = scaler_cbins.transform(cats_train_bins)
#cats_test_bins = scaler_cbins.transform(cats_test_bins)

# computed on full label set DOGS

# plot result vectors...

clf = mord.LogisticAT(alpha=1) # absolute loss - 
clf.fit(dogs_train, dogs_train_labels.ravel())
print('Mean Absolute Error of LogisticAT on Training Data %s' % metrics.mean_absolute_error(clf.predict(dogs_train), dogs_train_labels))
print('Mean Absolute Error of LogisticAT on Test Data %s' % metrics.mean_absolute_error(clf.predict(dogs_test), dogs_test_labels))

clf2 = mord.LogisticIT(alpha=1) # 0-1 loss
clf2.fit(dogs_train, dogs_train_labels.ravel())
print('Mean Absolute Error of LogisticIT on Training Data %s' % metrics.mean_absolute_error(clf2.predict(dogs_train), dogs_train_labels))
print('Mean Absolute Error of LogisticIT on Test Data %s' % metrics.mean_absolute_error(clf2.predict(dogs_test), dogs_test_labels))

clf3 = mord.LogisticSE(alpha=1) # squared error
clf3.fit(dogs_train, dogs_train_labels.ravel())
print('Mean Absolute Error of LogisticSE on Training Data %s' % metrics.mean_absolute_error(clf3.predict(dogs_train), dogs_train_labels))
print('Mean Absolute Error of LogisticSE on Test Data %s' % metrics.mean_absolute_error(clf3.predict(dogs_test), dogs_test_labels))

# CATS

clf = mord.LogisticAT(alpha=1)
clf.fit(cats_train, cats_train_labels.ravel())
print('Mean Absolute Error of LogisticAT on Training Data %s' % metrics.mean_absolute_error(clf.predict(cats_train), cats_train_labels))
print('Mean Absolute Error of LogisticAT on Test Data %s' % metrics.mean_absolute_error(clf.predict(cats_test), cats_test_labels))


clf2 = mord.LogisticIT(alpha=1)
clf2.fit(cats_train, cats_train_labels.ravel())
print('Mean Absolute Error of LogisticIT on Training Data %s' % metrics.mean_absolute_error(clf2.predict(cats_train), cats_train_labels))
print('Mean Absolute Error of LogisticIT on Test Data %s' % metrics.mean_absolute_error(clf2.predict(cats_test), cats_test_labels))


clf3 = mord.LogisticSE(alpha=1)
clf3.fit(cats_train, cats_train_labels.ravel())
print('Mean Absolute Error of LogisticSE on Trianing Data %s' % metrics.mean_absolute_error(clf3.predict(cats_train), cats_train_labels))
print('Mean Absolute Error of LogisticSE on Test Data %s' % metrics.mean_absolute_error(clf3.predict(cats_test), cats_test_labels))

temp_df = pd.DataFrame(dogs_train_labels)
temp_sub = temp_df.loc[temp_df[0]==1] #3212 rows out of 4487 hmmm
temp_sub.head() 

# output for binary classification
# before cv
#logistic = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
#logistic.fit(dogs_train,dogs_train_2_labels.ravel())

#dogs_test_labels = dogs_test_2_labels.reshape(dogs_test_2_labels.shape[0],)
#train_pred = logistic.predict(dogs_train_2)

#adopt_pred = logistic.predict(dogs_test_2)

#train_accuracy = metrics.accuracy_score(dogs_train_2_labels, train_pred)
#print('train accuracy (before cv): ', train_accuracy)

#test_accuracy = metrics.accuracy_score(dogs_test_2_labels, adopt_pred)
#print('test accuracy (before cv): ', test_accuracy)

#f1 = f1_score(dogs_test_2_labels, adopt_pred, average='macro')
#print(f1)

# before cv
logistic = linear_model.LogisticRegression(solver='lbfgs', multi_class='ovr')
logistic.fit(cats_train_2,cats_train_2_labels.ravel())

cats_test_labels = cats_test_2_labels.reshape(cats_test_2_labels.shape[0],)
train_pred = logistic.predict(cats_train_2)

adopt_pred = logistic.predict(cats_test_2)

train_accuracy = metrics.accuracy_score(cats_train_2_labels, train_pred)
print('train accuracy (before cv): ', train_accuracy)

test_accuracy = metrics.accuracy_score(cats_test_2_labels, adopt_pred)
print('test accuracy (before cv): ', test_accuracy)

f1 = f1_score(cats_test_2_labels, adopt_pred, average='macro')
print(f1)

In [29]:
# TRAINING VIA GRADIENT BOOST W/O BREED - DOGS
gbc = GradientBoostingClassifier()

parameters = {'learning_rate':[0.05, 0.1, 0.25, 0.5, 0.75, 1], 'max_depth': [1, 2, 3, 4, 5], 'min_samples_split':[2, 3, 4, 5]}
clf = GridSearchCV(gbc, parameters, cv=5)

clf.fit(dogs_train,dogs_train_labels.ravel())

train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
dt_btrain_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
dt_btest_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', dt_btrain_accuracy_dogs)
print('test accuracy (after cv): ', dt_btest_accuracy_dogs)
dt_bf1_dogs = f1_score(dogs_test_labels, adopt_pred_cv, average='macro')
print(dt_bf1_dogs)
print(clf.best_params_)


ValueError: Number of features of the model must match the input. Model n_features is 16 and input n_features is 323 

In [82]:
train_pred_cv = clf.predict(dogs_train)
adopt_pred_cv = clf.predict(dogs_test)
dt_btrain_accuracy_dogs = clf.score(dogs_train, dogs_train_labels)
dt_btest_accuracy_dogs = clf.score(dogs_test, dogs_test_labels)

print('train accuracy (after cv): ', dt_btrain_accuracy_dogs)
print('test accuracy (after cv): ', dt_btest_accuracy_dogs)
dt_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
print(dt_bf1_dogs)
print(clf.best_params_)

train accuracy (after cv):  0.7061716252766167
test accuracy (after cv):  0.7219251336898396
0.48123752245437235
{'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 5}


In [92]:
gbc = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, min_samples_split=3);
gbc.fit(dogs_train,dogs_train_labels.ravel())
train_pred_cv = gbc.predict(dogs_train);
adopt_pred_cv = gbc.predict(dogs_test);
np.savetxt("test-pred-gbc.csv", adopt_pred_cv, delimiter=",")
dt_btrain_accuracy_dogs = gbc.score(dogs_train, dogs_train_labels)
dt_btest_accuracy_dogs = gbc.score(dogs_test, dogs_test_labels)
print('train accuracy (after cv): ', dt_btrain_accuracy_dogs)
print('test accuracy (after cv): ', dt_btest_accuracy_dogs)
# dt_bf1_dogs = f1_score(dogs_breed_test_labels, adopt_pred_cv, average='macro')
dt_bf1_dogs = f1(dogs_test_labels, adopt_pred_cv);
print(dt_bf1_dogs)
print(gbc.feature_importances_)
print(dogs_sub_train_df.columns)

train accuracy (after cv):  0.8163265306122449
test accuracy (after cv):  0.7299465240641712
0.3637795275590551
[0.38275743 0.02263808 0.09142107 0.05965199 0.04180254 0.04727271
 0.09372869 0.02566839 0.13965228 0.02288722 0.02311444 0.0123933
 0.01171473 0.0089069  0.00499024 0.01139999]
Index(['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength',
       'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt'],
      dtype='object')


In [72]:
# TRAINING VIA GRADIENT BOOST W/O BREED - DOGS
gbc = GradientBoostingClassifier()

parameters = {'learning_rate':[0.05, 0.1, 0.25, 0.5, 0.75, 1], 'max_depth': [1, 2, 3, 4, 5], 'min_samples_split':[2, 3, 4, 5]}
clf = GridSearchCV(gbc, parameters, cv=5)

clf.fit(cats_train,cats_train_labels.ravel())

train_pred_cv = clf.predict(cats_train)
adopt_pred_cv = clf.predict(cats_test)
dt_btrain_accuracy_cats = clf.score(cats_train, cats_train_labels)
dt_btest_accuracy_cats = clf.score(cats_test, cats_test_labels)

print('train accuracy (after cv): ', dt_btrain_accuracy_cats)
print('test accuracy (after cv): ', dt_btest_accuracy_cats)
dt_bf1_cats = f1_score(cats_test_labels, adopt_pred_cv, average='macro')
print(dt_bf1_cats)
print(clf.best_params_)

train accuracy (after cv):  0.7765273311897106
test accuracy (after cv):  0.7469975980784628
0.5064086931618512
{'learning_rate': 0.05, 'max_depth': 3, 'min_samples_split': 5}


In [71]:
gbc = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, min_samples_split=5);
gbc.fit(cats_train,cats_train_labels.ravel())
print(gbc.feature_importances_)
print(cats_sub_train_df.columns)

[5.50508306e-01 3.26181793e-03 2.46817278e-02 3.93101061e-02
 3.27422173e-02 4.82047777e-02 2.09335052e-02 2.47222707e-04
 2.05357596e-01 3.73397810e-02 3.82639004e-03 1.26511702e-03
 5.27733426e-03 8.30683347e-03 8.48865314e-03 1.02486145e-02]
Index(['Age', 'Breed1', 'Gender', 'Color1', 'MaturitySize', 'FurLength',
       'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'PhotoAmt'],
      dtype='object')


In [91]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        y_true = [1 - x for x in y_true];
        y_pred = [1 - x for x in y_pred];
        mult = [x*y for x,y in zip(y_true, y_pred)];
        true_positives = np.sum(mult);
        possible_positives = np.sum(y_true);
        recall = true_positives / (possible_positives + .00000000000000001);
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        y_true = [1 - x for x in y_true];
        y_pred = [1 - x for x in y_pred];
        mult = [x*y for x,y in zip(y_true, y_pred)];
        true_positives = np.sum(mult);
        predicted_positives = np.sum(y_pred);
        precision = true_positives / (predicted_positives + .00000000000000001);
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+.00000000000000001))